### Loading Data (데이터를 Colab으로 다운로드)
##### wget으로 다운받은 압축 파일을 해제하면 다음과 같은 이미지(images)와 정답(labels) 데이터가 있는 test, train, valid 디렉토리 경로, 클래스 개수와 이름이 저장되어 있는 data.yaml 파일이 있는데, yaml 파일에서 커스텀 데이터로 학습(파인튜닝)할 때 필요한 train, valid 데이터가 저장되어 있는 train, val 디렉토리 경로는 반드시 본인의 환경에 맞게 변경해줘야 함.

In [ ]:
!wget -O Helmet_data.zip https://universe.roboflow.com/ds/ano3Pqyny3?key=05ZtQz93y6

In [ ]:
import zipfile

with zipfile.ZipFile('/content/Helmet_data.zip') as target_file:
    target_file.extractall('/content/Helmet_Data')

In [ ]:
!cat /content/Helmet_Data/data.yaml

### 커스텀 데이터에 맞는 YAML 파일 만들기
##### YOLOv8 학습과 검증에 필요한 train, valid 데이터의 디렉토리 경로와 Detection 하고싶은 클래스 개수 그리고 해당 클래스의 (문자열)이름이  저장되어 있는 YAML파일을 반드시 만들어 주어야 함.

In [ ]:
!pip install PyYAML

In [ ]:
import yaml

data = { 'train' : '/content/Helmet_Data/train/images/', # YOLOv8 학습과 검증에 사용되는 train, valid data가 저장되어 있는 디렉토리 경로
        'val' : '/content/Helmet_Data/valid/images/',
         'test' : '/content/Helmet_Data/test/images',
         'names' : ['HScooter'], # Detection 하고싶은 클래스의 개수(1개)와 클래스에 대응되는 클래스 이름(names)
         'nc' : 1}

with open('/content/Helmet_Data/Helmet_Data.yaml', 'w') as f: # 데이터 경로와 클래스 정보를 저장하고 있는 딕셔너리 객체 data를 YOLOv8 학습에 필요한 Helmet_Data.yaml 저장
    yaml.dump(data,f)

with open('/content/Helmet_Data/Helmet_Data.yaml', 'r') as f: # Helmet_Data.yaml 읽어서 화면에 출력
    helmet_yaml = yaml.safe_load(f)
    display(helmet_yaml)

#### Install YOLOv8

In [ ]:
!pip install ultralytics

In [ ]:
import ultralytics

ultralytics.checks()

#### Load a pre-trained model

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt') # MS COCO dataset 사전 학습된 yolov8n 모델을 로드함.

In [ ]:
print(type(model.names), len(model.names))
print(model.names)

#### YOLOv8 커스텀 데이터 학습하기

In [ ]:
#  epochs - 전체 데이터셋이 모델에 한 번 전달되는 횟수
#  patience - 조기종료 기법에서 사용되는 값. 성능이 향상되지 않는 epoch 횟수
#  batch - 한 번에 모델이 학습하는 데이터의 샘플 수. 작은 배치 크기는 메모리 사용량을 줄이고 학습속도를 높임. 큰 배치 크기는 더 안정적인 gradient update를 제공
#  imagsz - 모델이 입력되는 이미지 크기

# 전체 데이터셋이 10번 반복되면서 모델 학습
# 5번의 epoch동안 성능이 향상되지 않으면 학습을 조기종료
# 8개의 이미지를 동시에 모델에 전달
# 모델에 입력되는 이미지는 416X416 크기로 조정됨
model.train(data = '/content/Helmet_Data/Helmet_Data.yaml', epochs=10, patience=5, batch=8, imgsz=416)


In [ ]:
# Helmet_Data.yaml에 기술되어 있는 커스텀 데이터로 학습되었기 떄문에, 학습을 마친 후에 model.names 값을 보면, 사전학습된 MS COCO 데이터의 80개가 아닌 우리가 YAML 파일에서 설정한 1개의 클래스와 이름으로 바뀌어 있는 것을 알 수 있음.
print(type(model.names), len(model.names))

print(model.names)

#### 학습된 YOLOv8 이용해서 테스트 이미지 예측

In [ ]:
results = model.predict(source='/content/Helmet_Data/test/images/', save=True)